<a href="https://colab.research.google.com/github/m-numan1/100-Deep-Learning-Projects/blob/main/CIFAR10_Resnet_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cifar10 With Pre Trained Resnet model(Transfer Learning)
**Steps**
 * We Will following steps
 * Download datasets from torchvision.dataset
 * Import Libraries
 * create batches of data with DataLoader
 * Make NeuralNetwork
 * Train Loop
 * Evaluation Loop


In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [2]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

In [3]:
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
train_data = CIFAR10(root='./data',train=True,download=True,transform=transforms)
test_data = CIFAR10(root='./data',train=False,download=True,transform=transforms)

100%|██████████| 170M/170M [00:06<00:00, 26.4MB/s]


In [5]:
train_dataloader = DataLoader(train_data,batch_size=64,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=64,shuffle=True)

In [18]:
from torchvision import models
resnet = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Freezing the feature part of resnet50
for param in resnet.parameters():
  param.requires_grad = False

In [20]:
# adding classifier layer
resnet.fc = nn.Sequential(
    nn.Linear(2048,512),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(512,10)
)

for param in resnet.fc.parameters():
  param.requires_grad = True

In [21]:
epoch = 4
lr = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

optimizer = optim.Adam(resnet.fc.parameters(),lr=lr)
loss_fn = nn.CrossEntropyLoss()

resnet.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
# Training loop

for epoch in range(5):
  total_loss = 0
  for image_batch, label_batch in train_dataloader:
    image_batch = image_batch.to(device)
    label_batch = label_batch.to(device)

    optimizer.zero_grad()
    output = resnet(image_batch)
    loss = loss_fn(output, label_batch)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch: {epoch+1}, Loss: {total_loss/len(train_dataloader)}")



Epoch: 1, Loss: 0.865249138887581
Epoch: 2, Loss: 0.7169430135079967
Epoch: 3, Loss: 0.6867608395607575
Epoch: 4, Loss: 0.6708069224567974
Epoch: 5, Loss: 0.6577380695916197


In [23]:
# Evaluation loop

resnet.eval()

accuracy = 0
total = 0

with torch.no_grad():
  for image_batch, label_batch in test_dataloader:

    image_batch = image_batch.to(device)
    label_batch = label_batch.to(device)

    output = resnet(image_batch)

    _, predicted = torch.max(output.data, 1)
    total += label_batch.size(0)
    accuracy += (predicted == label_batch).sum().item()

  print(f"Test Accuracy: {accuracy/total}")

Test Accuracy: 0.8104
